## Padrón Comunidad de Madrid

https://datos.madrid.es/FWProjects/egob/Catalogo/Demografia/Padron/conceptos_fundamentales.pdf

1. Leer el CSV del padrón y pasarle un schema indicando el tipo de dato de cada columna. Una vez leído el CSV guardarlo en formato parquet


In [1]:
#First mport pyspark library and create a spark session

import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV to Dataset").getOrCreate()


In [2]:
#Leer el fichero csv sin schema. Con la opción inferSchema=True detecta automáticamente el tipo de cada columna.

df = spark.read.options(header=True, delimiter=';', quota='', inferSchema=True) \
    .csv('padron_madrid/Rango_Edades_Seccion_202011.csv')
df.printSchema()
df.show(5)

root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+-

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [4]:
#Leer el fichero con el schema indicado

schema = StructType() \
    .add('COD_DISTRITO', IntegerType(), True) \
    .add('DESC_DISTRITO', StringType(), True) \
    .add('COD_DIST_BARRIO', IntegerType(), True) \
    .add('DESC_BARRIO', StringType(), True) \
    .add('COD_BARRIO', IntegerType(), True) \
    .add('COD_DIST_SECCION', IntegerType(), True) \
    .add('COD_SECCION', IntegerType(), True) \
    .add('COD_EDAD_INT', IntegerType(), True) \
    .add('EspanolesHombres', IntegerType(), True) \
    .add('EspanolesMujeres', IntegerType(), True) \
    .add('ExtranjerosHombres', IntegerType(), True) \
    .add('ExtranjerosMujeres', IntegerType(), True)

df_schema = spark.read.format('csv') \
    .option('header',True) \
    .option('delimiter', ';') \
    .schema(schema) \
    .load('padron_madrid/Rango_Edades_Seccion_202011.csv')

df_schema.printSchema()
df_schema.show(5)

root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+-

In [6]:
#Seleccionamos las columnas numéricas y rellenamos valores nulos con 0

df_4campos = df_schema.select('EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres')
df_4campos.na.fill(0).show(5)

+----------------+----------------+------------------+------------------+
|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+----------------+----------------+------------------+------------------+
|               5|               0|                 1|                 0|
|               2|               3|                 0|                 5|
|               6|               1|                 1|                 0|
|               2|               0|                 0|                 3|
|               3|               3|                 0|                 0|
+----------------+----------------+------------------+------------------+
only showing top 5 rows



### Ejemplo de como crear una vista para hacer consultas sql

In [7]:
df_4campos_0 = df_4campos.na.fill(0)

#crear una vista para hacer consultas en sql
df_4campos_0.createOrReplaceTempView("people")

#spark.sql('SELECT * FROM people').show(5)

spark.sql('SELECT EspanolesHombres,EspanolesMujeres FROM people').show(5)


+----------------+----------------+
|EspanolesHombres|EspanolesMujeres|
+----------------+----------------+
|               5|               0|
|               2|               3|
|               6|               1|
|               2|               0|
|               3|               3|
+----------------+----------------+
only showing top 5 rows

